### <span style="color:GoldenRod ">**User Engagement Analysis Database**</span>

In [2]:
import pandas as pd
import json 
from sqlalchemy import create_engine

In [2]:
file_paths = {
    'business': r'F:\Portfolio Projects\User Engagement Analysis\yelp_dataset\yelp_academic_dataset_business.json',
    'checkin': r"F:\Portfolio Projects\User Engagement Analysis\yelp_dataset\yelp_academic_dataset_checkin.json",
    'review': r'F:\Portfolio Projects\User Engagement Analysis\yelp_dataset\yelp_academic_dataset_review.json',
    'tip': r"F:\Portfolio Projects\User Engagement Analysis\yelp_dataset\yelp_academic_dataset_tip.json",
    'user': r"F:\Portfolio Projects\User Engagement Analysis\yelp_dataset\yelp_academic_dataset_user.json"
}


In [ ]:
# Comsumed too much resources for loading the dataset

# with open(file_paths['business'], 'r') as f:
#     business_data = [json.loads(line) for line in f]
# business_df = pd.DataFrame(business_data)

# with open(file_paths['checkin'], 'r') as f:
#     checkin_data = [json.loads(line) for line in f]
# checkin_df = pd.DataFrame(checkin_data)

# with open(file_paths['review'], 'r') as f:
#     review_data = [json.loads(line) for line in f]
# review_df = pd.DataFrame(review_data)

# with open(file_paths['tip'], 'r') as f:
#     tip_data = [json.loads(line) for line in f]
# tip_df = pd.DataFrame(tip_data)

# with open(file_paths['user'], 'r') as f:
#     user_data = [json.loads(line) for line in f]
# user_df = pd.DataFrame(user_data)

In [24]:
# # Create engine for SQLlite 
# engine = create_engine('sqlite:///yelp.db')

# # Function to load DataFrame into SQL
# def load_dataframe(df, table_name, engine):
#     df.to_sql(table_name, con=engine, if_exists='replace', index=False)

# # Load each DataFrame into a separate table
# load_dataframe(business_df, 'business', engine)
# load_dataframe(review_df, 'review', engine)
# load_dataframe(user_df, 'user', engine)
# load_dataframe(tip_df, 'tip', engine)
# load_dataframe(checkin_df, 'checkin', engine)

In [21]:
# Example query to check loaded data
df = pd.read_sql('SELECT business_id, attributes, hours FROM business LIMIT 5', con=engine)
df

# Convert JSON strings back to Python objects if needed
df['attributes'] = df['attributes'].apply(lambda x: json.loads(x) if x else {})
print(df['attributes'])

0                        {'ByAppointmentOnly': 'True'}
1               {'BusinessAcceptsCreditCards': 'True'}
2    {'BikeParking': 'True', 'BusinessAcceptsCredit...
3    {'RestaurantsDelivery': 'False', 'OutdoorSeati...
4    {'BusinessAcceptsCreditCards': 'True', 'Wheelc...
Name: attributes, dtype: object


In [29]:
# To delete the database
# import os 

# # Delete the existing database file
# db_path = 'yelp_optimized.db'
# if os.path.exists(db_path):
#     os.remove(db_path)
#     print(f"Database '{db_path}' deleted successfully.")
# else:
#     print(f"Database '{db_path}' does not exist.")

Database 'yelp_optimized.db' deleted successfully.


In [32]:
import json
import pandas as pd
from sqlalchemy import create_engine

# Create a database connection using SQLite
engine = create_engine('sqlite:///yelp_optimized.db')

# Function to handle preprocessing of nested fields (like dictionaries or lists)
def preprocess_chunk(chunk):
    # Drop unwanted columns
    chunk = chunk.drop(columns=['attributes', 'hours'], errors='ignore')

    for col in chunk.columns:
        if chunk[col].dtype == 'object':    # Check if the column contains object data type
            # Convert dictionaries or lists to JSON strings for storage in the database
            chunk[col] = chunk[col].apply(lambda x: json.dumps(x) if isinstance(x, (dict, list)) else x)
    return chunk

# Function to process large JSON files in manageable chunks
def process_json_file(file_path, table_name, chunk_size=10000):
    """
    Reads a large JSON file line by line, processes it in chunks, 
    and loads the data into a database table.

    Parameters:
    - file_path: Path to the JSON file
    - table_name: Name of the database table to insert data into
    - chunk_size: Number of rows to process at a time
    """
    print(f"Processing {file_path} for table {table_name}")     # Log progress
    chunk = []      # Temporary storage for rows in the current chunk

    # Open the JSON file and read it line by line
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            # Parse each line as JSON and append it to the chunk
            chunk.append(json.loads(line))
            
            # If the chunk reaches the specified size, process and load it into the database
            if len(chunk) >= chunk_size:
                df = pd.DataFrame(chunk)    # Convert the chunk to a DataFrame
                df = preprocess_chunk(df)  # Preprocess nested fields and drop columns
                df.to_sql(table_name, con=engine, if_exists='append', index=False)
                chunk = []  # Clear the chunk after processing
                print(f"Processed {i+1} rows into {table_name}")    # Log progress
        
        # After the loop, process any remaining rows in the chunk
        if chunk:
            df = pd.DataFrame(chunk)
            df = preprocess_chunk(df)  # Preprocess nested fields and drop columns
            df.to_sql(table_name, con=engine, if_exists='append', index=False)      # Load to database
            print(f"Processed {len(chunk)} remaining rows into {table_name}")       # Log progress

# Define file paths for the JSON datasets
file_paths = {
    'business': r'F:\Portfolio Projects\User Engagement Analysis\yelp_dataset\yelp_academic_dataset_business.json',
    'checkin': r"F:\Portfolio Projects\User Engagement Analysis\yelp_dataset\yelp_academic_dataset_checkin.json",
    'review': r'F:\Portfolio Projects\User Engagement Analysis\yelp_dataset\yelp_academic_dataset_review.json',
    'tip': r"F:\Portfolio Projects\User Engagement Analysis\yelp_dataset\yelp_academic_dataset_tip.json",
    'user': r"F:\Portfolio Projects\User Engagement Analysis\yelp_dataset\yelp_academic_dataset_user.json"
}

# Process each JSON dataset and load it into the respective database table
process_json_file(file_paths['business'], 'business')
process_json_file(file_paths['checkin'], 'checkin')
process_json_file(file_paths['review'], 'review')
process_json_file(file_paths['tip'], 'tip')
process_json_file(file_paths['user'], 'user')

# Print a message once all datasets have been processed
print("All datasets processed and loaded into the database.")


Processing F:\Portfolio Projects\User Engagement Analysis\yelp_dataset\yelp_academic_dataset_business.json for table business
Processed 10000 rows into business
Processed 20000 rows into business
Processed 30000 rows into business
Processed 40000 rows into business
Processed 50000 rows into business
Processed 60000 rows into business
Processed 70000 rows into business
Processed 80000 rows into business
Processed 90000 rows into business
Processed 100000 rows into business
Processed 110000 rows into business
Processed 120000 rows into business
Processed 130000 rows into business
Processed 140000 rows into business
Processed 150000 rows into business
Processed 346 remaining rows into business
Processing F:\Portfolio Projects\User Engagement Analysis\yelp_dataset\yelp_academic_dataset_checkin.json for table checkin
Processed 10000 rows into checkin
Processed 20000 rows into checkin
Processed 30000 rows into checkin
Processed 40000 rows into checkin
Processed 50000 rows into checkin
Proces

In [6]:
engine = create_engine('sqlite:///yelp_optimized.db')

In [7]:
# Inspect the table structure
df = pd.read_sql("select * from business limit 5;", con=engine)
df.head()


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,"Doctors, Traditional Chinese Medicine, Naturop..."
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,"Shipping Centers, Local Services, Notaries, Ma..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"Department Stores, Shopping, Fashion, Home & G..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"Brewpubs, Breweries, Food"


In [17]:
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(tables_query, con=engine)
print(tables)

       name
0  business
1   checkin
2    review
3       tip
4      user


In [15]:
business_df = pd.read_sql("SELECT * FROM business LIMIT 5;", con=engine)
checkin_df = pd.read_sql("SELECT * FROM checkin LIMIT 5;", con=engine)
review_df = pd.read_sql("SELECT * FROM review LIMIT 5;", con=engine)
tip_df = pd.read_sql("SELECT * FROM tip LIMIT 5;", con=engine)
user_df = pd.read_sql("SELECT * FROM user LIMIT 5;", con=engine)

In [16]:
# Check shapes
print(business_df.shape)
print(checkin_df.shape)
print(review_df.shape)
print(tip_df.shape)
print(user_df.shape)

(5, 12)
(5, 2)
(5, 9)
(5, 5)
(5, 22)
